In [1]:
import pandas as pd
import numpy as np
import json

import networkx as nx

from tqdm import tqdm

from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# create class instances
data = Data()
graph = Graph()

# read data from csv
cpds = pd.read_csv('data/compounds_final.csv', index_col=0)
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)


# number of times a metabolite apperas on pairs dataset
graph.get_number_of_occurences(pairs)

# Create Graph
graph.create_graph(data=data, pairs=pairs)

''' 
*******************************************
Validate the methods on validation datasets 
*******************************************
'''
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))

paths = graph.simple_weighted_shortest_path(test_cases=test_cases, data=data, method='mol_weight')

# ######### NEW VALIDATION SET ###########
# pyminer_test = pd.read_csv('data/original/pyminer_validation_set.csv', delimiter=';', header=None, names=['Pathway'])
# pyminer_test['source'] = pyminer_test['Pathway'].apply(lambda x: x.split(',')[0])
# pyminer_test['target'] = pyminer_test['Pathway'].apply(lambda x: x.split(',')[len(x.split(','))-1])
# pyminer_test['paths_list'] = pyminer_test['Pathway'].apply(lambda x: x.split(','))

# print('Simple weighted shortes paths:')
# paths = graph.simple_weighted_shortest_path(test_cases=pyminer_test, data=data, method='mol_weight')

# nodes: 8591 
# edges: 30081

Removing self-loops...
# nodes: 8591 
# edges: 30026


100%|██████████| 30026/30026 [00:00<00:00, 607375.87it/s]


Correct pathway predictions: 14
Correct pathway predictions (%): 28.0


### Save predicted paths

In [5]:
import os

directory = 'data/results'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

paths.to_csv('data/results/predicted_paths.csv')

# Graph learning and stuff to try

In [ ]:
cpd = data.get_compound_by_id('C00082')
print(cpd)

smile = cpd.smiles
print(smile)

correct_pathway_example = paths['Pathway'].iloc[1]
print(correct_pathway_example)

correct_subgraph = graph.G.subgraph(correct_pathway_example)
print(correct_subgraph.nodes())

## GNN! (Maybe good)

In [118]:
gg = graph.G

AttributeError: 'Graph' object has no attribute 'G'

In [126]:
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric
import torch

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Global mean pooling to obtain graph-level representation
        x = torch_geometric.nn.global_mean_pool(x, batch)
        
        # apply a final classifier
        x = self.fc(x)
        
        return x

In [127]:
master_G = gg.copy()
print(master_G)

Graph with 8591 nodes and 30026 edges


In [128]:
graphs = []
labels = []

for row in range(len(paths)):
    sg = master_G.subgraph(paths['Pathway'].iloc[row])
    graphs.append(sg)

    if paths['Correct'].iloc[row]: label = 1
    else: label = 0

    labels.append(label)

print("Graphs:", graphs)
print("Labels:", labels)

Graphs: [<networkx.classes.graph.Graph object at 0x7f3761267fa0>, <networkx.classes.graph.Graph object at 0x7f375ba9d430>, <networkx.classes.graph.Graph object at 0x7f375ba9db50>, <networkx.classes.graph.Graph object at 0x7f375bb11be0>, <networkx.classes.graph.Graph object at 0x7f375bb110a0>, <networkx.classes.graph.Graph object at 0x7f375bb11b80>, <networkx.classes.graph.Graph object at 0x7f375bb11a30>, <networkx.classes.graph.Graph object at 0x7f375bb11dc0>, <networkx.classes.graph.Graph object at 0x7f375bb11ee0>, <networkx.classes.graph.Graph object at 0x7f37ff0a3310>, <networkx.classes.graph.Graph object at 0x7f37ff0a3760>, <networkx.classes.graph.Graph object at 0x7f37ff0a39a0>, <networkx.classes.graph.Graph object at 0x7f37ff0a3cd0>, <networkx.classes.graph.Graph object at 0x7f37ff0a3a00>, <networkx.classes.graph.Graph object at 0x7f37ff0a32e0>, <networkx.classes.graph.Graph object at 0x7f37ff0a3610>, <networkx.classes.graph.Graph object at 0x7f37ff0a3250>, <networkx.classes.grap

In [145]:
# Convert NetworkX graphs to PyG data
data_list = []
for graph, label in zip(graphs, labels):
    # rename nodes to integers    
    mapping = {node: idx for idx, node in enumerate(graph.nodes())}
    graph = nx.relabel_nodes(graph, mapping)

    # Convert NetworkX graph to PyG data
    x = torch.tensor([graph.nodes[node]['mw'] for node in graph.nodes], dtype=torch.float).view(-1, 1)
    edge_index = torch.tensor(list(graph.edges)).t().contiguous()
    edge_attr = torch.tensor([graph.edges[edge]['mol_weight'] for edge in graph.edges], dtype=torch.float)
    y = torch.tensor([label])  # Graph-level label

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    data_list.append(data)

# Concatenate all data samples into a single PyG data object
data = torch_geometric.data.Batch.from_data_list(data_list)
print(data)

DataBatch(x=[306, 1], edge_index=[2, 256], edge_attr=[256], y=[50], batch=[306], ptr=[51])


In [146]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=.1)

In [156]:
import torch.optim as optim

# Set random seed for reproducibility
torch.manual_seed(42)

# Initialize GNN model
model = GCN(input_dim=data.x.shape[1], hidden_dim=128, output_dim=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the model
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    
    total_loss = 0
    for data in train_data:
        out = model(data)
        loss = criterion(out, data.y.view(-1))
        loss.backward()
        total_loss += loss.item()
    
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

print("Training completed.")

Epoch 0, Loss: 291.3220929327453
Epoch 10, Loss: 187.51650398398237
Epoch 20, Loss: 26.978775560855865
Epoch 30, Loss: 29.461252726614475
Epoch 40, Loss: 30.913760513067245
Epoch 50, Loss: 28.086616799235344
Epoch 60, Loss: 27.01753768324852
Epoch 70, Loss: 27.176073491573334
Epoch 80, Loss: 27.016084015369415
Epoch 90, Loss: 26.955724462866783
Training completed.


In [157]:
# Evaluation mode
model.eval()
with torch.no_grad():
    correct = 0
    total = 0

    for data in test_data:
        out = model(data)
        predicted_labels = out.argmax(dim=1)
        true_labels = data.y.view(-1)
        correct += (predicted_labels == true_labels).sum().item()
        total += len(true_labels)
        print(predicted_labels, data.y)

    accuracy = correct / total
    print(f"Accuracy: {accuracy}")

tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([1])
Accuracy: 0.8


# EXAMPLE OF GRAPHNN

In [113]:
import networkx as nx
import random

# Generate random graphs with labels
num_graphs = 5
graphs = []
labels = []

for _ in range(num_graphs):
    # Generate a random graph
    graph = nx.fast_gnp_random_graph(10, 0.3)
    graphs.append(graph)
    
    # Assign a random label (Type 0 or Type 1)
    label = random.choice([0, 1])
    labels.append(label)

print("Graphs:", graphs)
print("Labels:", labels)

Graphs: [<networkx.classes.graph.Graph object at 0x7f375a17dac0>, <networkx.classes.graph.Graph object at 0x7f375a17da30>, <networkx.classes.graph.Graph object at 0x7f375a17da00>, <networkx.classes.graph.Graph object at 0x7f375a17db80>, <networkx.classes.graph.Graph object at 0x7f375a17dfd0>]
Labels: [0, 0, 0, 1, 0]


In [114]:
from torch_geometric.data import Data
import torch_geometric
import torch

# Convert NetworkX graphs to PyG data
data_list = []
for graph, label in zip(graphs, labels):
    # Convert NetworkX graph to PyG data
    edge_index = torch.tensor(list(graph.edges)).t().contiguous()
    x = torch.randn(graph.number_of_nodes(), 16)  # Random node features (16 dimensions)
    y = torch.tensor([label])  # Graph-level label
    
    data = Data(x=x, edge_index=edge_index, y=y)
    data_list.append(data)


# Concatenate all data samples into a single PyG data object
data = torch_geometric.data.Batch.from_data_list(data_list)
data

DataBatch(x=[50, 16], edge_index=[2, 64], y=[5], batch=[50], ptr=[6])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Global mean pooling to obtain graph-level representation
        x = torch_geometric.nn.global_mean_pool(x, batch)

        return x


In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

In [ ]:
import torch.optim as optim

# Set random seed for reproducibility
torch.manual_seed(42)

# Initialize GNN model
model = GCN(input_dim=16, hidden_dim=32, output_dim=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the model
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    
    total_loss = 0
    for data in train_data:
        out = model(data)
        loss = criterion(out, data.y.view(-1))
        loss.backward()
        total_loss += loss.item()
    
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

print("Training completed.")

In [ ]:
# Evaluation mode
model.eval()
with torch.no_grad():
    correct = 0
    total = 0

    for data in test_data:
        out = model(data)
        predicted_labels = out.argmax(dim=1)
        true_labels = data.y.view(-1)
        correct += (predicted_labels == true_labels).sum().item()
        total += len(true_labels)

    accuracy = correct / total
    print(f"Accuracy: {accuracy}")